In [42]:
import os

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
import cv2
import innvestigate
import numpy as np
import matplotlib.pyplot as plt
tf.compat.v1.disable_eager_execution()

In [38]:
model = load_model("resnet50_2layers.h5")
model.layers[-3]._name = "dense_after_conv"
model.layers[-2]._name = "dense_mid"
model.layers[-1]._name = "output"
model = innvestigate.model_wo_output_activation(model)

In [39]:
def do_innvestigate(analyzer_name, model, input_image, path_to_save, cmap="seismic", parameters=None):
    analyzer = innvestigate.create_analyzer(analyzer_name, model, **parameters if parameters else {})
    image = cv2.resize(cv2.imread(input_image), (192, 192))
    x = preprocess_input(np.array([image]))
    a = analyzer.analyze(x)
    a = a.sum(axis=np.argmax(np.asarray(a.shape) == 3))
    a /= np.max(np.abs(a))
    plt.imshow(a[0], cmap=cmap, clim=(-1, 1))
    plt.savefig(path_to_save)

In [40]:
PATH_TO_INPUT = r"C:\Users\PC\OneDrive\Pulpit\baza danych\to_generate"
PATH_TO_OUTPUT = r"C:\Users\PC\OneDrive\Pulpit\baza danych\innvestigate"

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\deconvnet_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("deconvnet", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\deconvnet_"+folder+"\\"+file, cmap="gray")

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\guided_backprop_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("guided_backprop", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\guided_backprop_"+folder+"\\"+file, cmap="gray")

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\input_t_gradient_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("input_t_gradient", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\input_t_gradient_"+folder+"\\"+file)

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\lrp.z_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("lrp.z", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\lrp.z_"+folder+"\\"+file)

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\smoothgrad_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("smoothgrad", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\smoothgrad_"+folder+"\\"+file, parameters={"augment_by_n": 64, "noise_scale": (128 - -128) * 0.1, "postprocess": "square"})

In [ ]:
for folder in os.listdir(PATH_TO_INPUT):
    os.makedirs(PATH_TO_OUTPUT + "\gradient_"+folder)
    for file in os.listdir(PATH_TO_INPUT+"\\"+folder):
        do_innvestigate("gradient", model, PATH_TO_INPUT+"\\"+folder+"\\"+file, PATH_TO_OUTPUT+"\gradient_"+folder+"\\"+file, parameters={"postprocess": "abs"})